# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

## Задание 1. Напишите свою функцию prefilter_items

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def prefilter_items(data, take_n_popular=5000):
    """Предфильтрация товаров"""
    
    # 1. Удаление товаров, со средней ценой < 1$
    price_to_remove_1=1
    less_def_price_item_ids=\
        data.groupby('item_id').filter(lambda group: group['sales_value'].mean() < price_to_remove_1).\
        reset_index()['item_id']
    data=data.loc[~data['item_id'].isin(less_def_price_item_ids)]
    
    # 2. Удаление товаров со средней ценой > 30$
    price_to_remove_2=30
    more_def_price_item_ids=\
        data.groupby('item_id').filter(lambda group: group['sales_value'].mean() > price_to_remove_2).\
        reset_index()['item_id']
    data=data.loc[~data['item_id'].isin(more_def_price_item_ids)]

    
    # 3. Придумайте свой фильтр
    # Удаление товаров которые купили только 1 раз - непопулярных
    quantity_to_remove=1
    def_quantity_item_ids=\
        data.groupby('item_id').filter(lambda group: group['quantity'].sum() == quantity_to_remove).reset_index()['item_id']        
        
    data=data.loc[~data['item_id'].isin(def_quantity_item_ids)]
    
    # 4. Выбор топ-N самых популярных товаров (N = take_n_popular)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
    popularity.sort_values('user_id', ascending=False, inplace=True)
    top_popular=popularity.head(take_n_popular)['item_id']       
    data = data.loc[data['item_id'].isin(top_popular)]
    
    #data=data.groupby('item_id').nunique()
    #data.sort_values('user_id', ascending=False, inplace=True)
    
    return data

In [3]:
data = pd.read_csv('data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
#Проверка prefilter_items
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_test = data_test[data_test['user_id'].isin(data_train['user_id'])]

n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5000


В корне проекта создайте модуль *src*. Положите функцию *prefilter_items* в файл *src/utils.py*  

Далее запустите код ниже и перейдите к заданию 2 (в конце ноутбука)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [6]:
data = pd.read_csv('data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5000


In [10]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,397896,818980,818981,819063,819255,819304,819308,819330,819518,819594,...,15926844,15926863,15926885,15926886,15926887,15926927,15927403,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [13]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 4.19 s


-----

# Домашнее задание

1. Перенесите метрики из ДЗ 1 src/metrics.py
3. Создайте модуль src/recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src/recommenders.py
4. Проверьте, что все модули корректно импортируются

In [14]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight




----

Проверка, что все работает

In [15]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender



In [16]:
main_recommender=MainRecommender(data_train, item_features, weighting=False)

In [17]:
data_train['user_id']

0          2375
2          2375
4          2375
5          2375
6          2375
           ... 
2282318    1168
2282321     462
2282322     462
2282323     462
2282324     462
Name: user_id, Length: 1322253, dtype: int64

In [18]:
res=main_recommender.get_similar_items_recommendation(2375)
res

[868764, 840361, 6534178, 1029743, 995242]

In [19]:
res2=main_recommender.get_similar_users_recommendation(2375)
res2

[6534178,
 6534178,
 1029743,
 995242,
 1106523,
 1029743,
 6534178,
 6534178,
 6534178]